## Importing files

In [24]:
import keras
from keras.datasets import cifar10
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
import scipy
from scipy import misc
import os

## Load data

In [25]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

## Load the model

In [26]:
model = VGG16(weights='imagenet',include_top = False, input_shape = (64,64,3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

## Obtain bottleneck features for train dataset

In [27]:
if os.path.exists('vgg16_features_train.npz'):
    print('bottleneck features detected train')
    features = np.load('vgg16_features_train.npz')['features']
else:
    print('bottleneck features are calculated now')
    big_x_train = np.array([scipy.misc.imresize(x_train[i], (64, 64, 3)) 
                            for i in range(0, len(x_train))]).astype('float32')
    vgg16_input_train = preprocess_input(big_x_train)
    print('train data preprocessed')
    features = model.predict(vgg16_input_train)
    np.savez('vgg16_features_train',features = features)
print('bottleneck features saved for train')

bottleneck features detected train
bottleneck features saved for train


## Obtain bottleneck features for test dataset

In [28]:
if os.path.exists('vgg16_features_test.npz'):
    print('bottleneck features detected test')
    features_test = np.load('vgg16_features_test.npz')['features_test']
else:
    print('bottleneck features are calculated now')
    big_x_test = np.array([scipy.misc.imresize(x_test[i], (64, 64, 3)) 
                       for i in range(0, len(x_test))]).astype('float32')
    vgg16_input_test = preprocess_input(big_x_test)
    print('test data preprocessed')
    features_test = model.predict(vgg16_input_test)
    np.savez('vgg16_features_test',features_test = features_test)
print('bottleneck features saved for test')

bottleneck features detected test
bottleneck features saved for test


## Labels

In [29]:
for i in range(5):
    print(y_train[i])

[6]
[9]
[9]
[4]
[1]


## Reshaping datasets

In [30]:
print(features.shape)
print(features_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 2, 2, 512)
(10000, 2, 2, 512)
(50000, 1)
(10000, 1)


In [31]:
features=features.reshape(50000,2*2*512)
features_test=features_test.reshape(10000,2*2*512)
y_train = y_train.flatten()
y_test=y_test.flatten()

In [32]:
print(features.shape)
print(features_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 2048)
(10000, 2048)
(50000,)
(10000,)


## Define the model architecture

In [33]:
features.shape[1:]

(2048,)

In [34]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(features, y_train)
y_pred = gnb.predict(features_test)

## Evaluate model classification accuracy on test set

In [35]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100)

Accuracy: 59.69
